# 基于词性分析的地址解析匹配算法

In [1]:
from paddlenlp import Taskflow

/bin/sh: 1: cat: not found
/bin/sh: 1: grep: not found
/bin/sh: 1: cat: not found
/bin/sh: 1: grep: not found
Error: Your machine doesn't support AVX, but the installed PaddlePaddle is avx core, you should reinstall paddlepaddle with no-avx core.


ModuleNotFoundError: No module named 'paddle.fluid.core_noavx'

In [ ]:
tag = Taskflow("pos_tagging",user_dict="user_dict.txt")

In [ ]:
data = tag("山东省日照市秦楼街道城建花园社区济南路壹号学府1号楼-1-1201")
print(data)

In [ ]:
tag.kwargs

## 移除省市数据信息

先将识别出来小区的信息数据，将前面的省市数据去掉，只保留小区数据


In [ ]:
dict = []
str = []
for item in data:
    dict.append(item[1])
    str.append(item[0])
print(dict)

In [ ]:
tag = 'ADDRESS'
tagIndex = -1
list = []
data = []
for i,item in enumerate(dict):
    if item == tag:
        tagIndex = i
    if tagIndex!=-1:
        list.append(item)
        data.append(str[i])

print(list)

## 读取matcher数据匹配文件

In [ ]:

import sys
result = []
with open('matchers.txt','r') as f:
    for line in f:    
        if not line.startswith('#') and len(line.strip('\n'))>0:
            result.append(line.strip('\n'))
print(result)


## 定义数据匹配与数据解析提取函数

In [ ]:
def match(match,list):
    str = ",".join(list)
    array =  match.split("=>")
    m = array[0]
    return m==str,array[1];

print(match(result[0],list)) 
# print(match('ADDRESS,LOC,w,m,w,m,m2',list) 

In [ ]:
def parser(matcher,list,data):
    
    res = []
   
    for i,m in enumerate(matcher.split(",")):
        flags = m.split("_");
        val = "null"
        index = 1
        if len(flags) == 2 :
            f = flags[0]
            n = int(flags[1])           
            for j,item in enumerate(list):    
                if item == f:                   
                    if index == n:
                        val = data[j]                                    
                        res.append(val)
                    index = index +1
        else:
             res.append(val)
    return res 



## 数据测试演示

In [ ]:
def execute(list,data):
    for item in result:
        flag, matcher = match(item,list)
        if flag:
           res =  parser(matcher,list,data)
           return res 

output = execute(list,data)
print('原数据:',data)        
print('词性:',list)
print('结果:',output)